In [ ]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
! pip install ray
! pip install tensorboardX==2.5.1
! pip install tensorboard==2.11.2
from google.colab import drive, files
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
checkpoint_path = repository_path + 'models/preprocessing/'
checkpoint_archive = '/content/' + checkpoint_path.split('/')[-2] + '.zip'

sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main__fastscnn_hyperparameters_tuning'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [1]:
import os
import torch
from torch import nn, optim
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.cloud.UNet import unet
from metrics_and_losses import metrics
from utils import segmentation_labels, custom_transforms, model_names
from models import config
from slurm_scripts import slurm_config
from ray import tune
from ray.tune import CLIReporter
from functools import partial

try:
  executing_on_colab
except NameError:
  executing_on_colab = False
  weights_path = config.WEIGHTS_PATH
  dataset_path = config.DATASET_PATH
  checkpoint_path = config.PREPROCESSING_PATH

In [2]:
# === defining transforms ===

diameter = 7
sigma_color = 50
sigma_space = 100
horizontal_flip = custom_transforms.PartiallyDeterministicHorizontalFlip(p=0.5)
center_crop = custom_transforms.PartiallyDeterministicCenterCrop(p=0.5)

image_transform = T.Compose([
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path, file_name=config.DATASET_INDEX_NAME)
X_train, _, Y_train, _ = train_test_split(img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.CcncsaDataset(X_train, Y_train, image_transform, target_transform)

# model parameters
model_name = "unet"
model = unet.UNet(out_channels=n_classes)
optimizer = optim.Adam

# setting up hyperparameters for training
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
n_epochs = 10
batch_size = 32
loss_fn = nn.CrossEntropyLoss()
score_fn = metrics.batch_mIoU
learning_rate = 0.01
class_weights = torch.tensor(config.CLASS_WEIGHTS, device=device)

# Ray Tune parameters
cpus_per_trial = 4
gpus_per_trial = 0  # torch.cuda.device_count()
num_samples = 1  # Number of times each combination is sampled (n_epochs are done per sample)
metric = "val_loss"
metrics_columns = ["train_loss", "train_score", "val_loss", "val_score", "training_iteration"]
local_dir = checkpoint_path + model_names.MODEL_NAMES[model_name]
max_report_frequency = 600
reporter = CLIReporter(
    metric_columns=metrics_columns, max_report_frequency=max_report_frequency)

In [3]:
# === training without augmentation ===

checkpoint_dir = local_dir if executing_on_colab else (os.path.abspath('./' + local_dir) + '/')
cfg = {
    "lr": learning_rate,
    "lr_scheduler": "none",
    "batch_size": batch_size,
    "transform": "none",
    "from_checkpoint": False,
    "checkpoint_dir": checkpoint_dir
    }

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

2023-02-02 22:54:17,698	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-02-02 22:54:27,486	WARNING worker.py:1851 -- Warning: The actor ImplicitFunc is very large (33 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


== Status ==
Current time: 2023-02-02 22:54:28 (running for 00:00:03.85)
Memory usage on this node: 9.9/11.9 GiB 
Using FIFO scheduling algorithm.
Resources requested: 4.0/4 CPUs, 0/1 GPUs, 0.0/1.36 GiB heap, 0.0/0.68 GiB objects
Result logdir: c:\Users\matte\Desktop\deep-seasonal-color-analysis-system\models\preprocessing\UNet\train_model_2023-02-02_22-54-23
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-----------------+
| Trial name              | status   | loc             |
|-------------------------+----------+-----------------|
| train_model_2753b_00000 | RUNNING  | 127.0.0.1:19960 |
+-------------------------+----------+-----------------+


== Status ==
Current time: 2023-02-02 23:04:32 (running for 00:10:07.68)
Memory usage on this node: 11.4/11.9 GiB : ***LOW MEMORY*** less than 10% of the memory on this node is available for use. This can cause unexpected crashes. Consider reducing the memory used by your application or reducing the Ray object store

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_2753b_00000,2023-02-03_05-40-18,False,,7ab8716633ae410dbca0b8fd09833a6b,DESKTOP-OQ33RCN,1,127.0.0.1,19960,24337.3,24337.3,24337.3,1675399218,0,,0.699404,0.337206,1,2753b_00000,0.702006,0.311795,0.0140116


== Status ==
Current time: 2023-02-03 05:45:49 (running for 06:51:25.41)
Memory usage on this node: 6.3/11.9 GiB 
Using FIFO scheduling algorithm.
Resources requested: 4.0/4 CPUs, 0/1 GPUs, 0.0/1.36 GiB heap, 0.0/0.68 GiB objects
Result logdir: c:\Users\matte\Desktop\deep-seasonal-color-analysis-system\models\preprocessing\UNet\train_model_2023-02-02_22-54-23
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-----------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc             |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-----------------+--------------+---------------+------------+-------------+----------------------|
| train_model_2753b_00000 | RUNNING  | 127.0.0.1:19960 |     0.699404 |      0.337206 |   0.702006 |    0.311795 |                    1 |
+-------------------------+----------+--------------

2023-02-03 07:50:31,254	WARNING tune.py:690 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


== Status ==
Current time: 2023-02-03 07:50:32 (running for 08:56:07.84)
Memory usage on this node: 9.0/11.9 GiB 
Using FIFO scheduling algorithm.
Resources requested: 4.0/4 CPUs, 0/1 GPUs, 0.0/1.36 GiB heap, 0.0/0.68 GiB objects
Result logdir: c:\Users\matte\Desktop\deep-seasonal-color-analysis-system\models\preprocessing\UNet\train_model_2023-02-02_22-54-23
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-----------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc             |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-----------------+--------------+---------------+------------+-------------+----------------------|
| train_model_2753b_00000 | RUNNING  | 127.0.0.1:19960 |     0.699404 |      0.337206 |   0.702006 |    0.311795 |                    1 |
+-------------------------+----------+--------------

2023-02-03 07:50:32,542	ERROR tune.py:758 -- Trials did not complete: [train_model_2753b_00000]
2023-02-03 07:50:32,545	INFO tune.py:762 -- Total run time: 32170.57 seconds (32167.82 seconds for the tuning loop).
2023-02-03 07:50:32,547	WARNING tune.py:768 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)

In [ ]:
# === training with augmentation: ColorJitter ===

image_transform_with_augmentation = T.Compose([
    T.ColorJitter(brightness=0.25, contrast=0.25), 
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform)
    
model = unet.UNet(out_channels=n_classes)

cfg["transform"] = "color_jitter"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)

In [ ]:
# === training with augmentation: horizontal_flip ===

image_transform_with_augmentation = T.Compose([
    horizontal_flip,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform_with_augmentation = T.Compose([
    horizontal_flip,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform_with_augmentation)
    
model = unet.UNet(out_channels=n_classes)

cfg["transform"] = "horizontal_flip"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)

In [ ]:
# === training with augmentation: center_crop ===

image_transform_with_augmentation = T.Compose([
    center_crop,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), 
    custom_transforms.BilateralFilter(sigma_color, sigma_space, diameter), 
    T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])

target_transform_with_augmentation = T.Compose([
    center_crop,
    T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

train_dataset_with_augmentation = dataset.CcncsaDataset(
    X_train, Y_train, image_transform_with_augmentation, target_transform_with_augmentation)

model = unet.UNet(out_channels=n_classes)

cfg["transform"] = "center_crop"

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset_with_augmentation, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {checkpoint_path}

In [ ]:
# command to clean the checkpoint directory in colab file system 
# ! rm -rf /content/deep-seasonal-color-analysis-system/models/preprocessing